In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
from sklearn.preprocessing import StandardScaler


In [19]:
# Configure TensorFlow to use GPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [15]:
#Dataset preprocessing for dataset-2

# Load your CSV data using pandas
data2 = pd.read_csv('cwru_bearing.csv')
import pandas as pd

# Step 1: Identify unique labels in the last column
unique_labels = data2['fault'].unique()

# Step 2: Create a mapping from labels to numbers
label_to_number = {label: index for index, label in enumerate(unique_labels)}

# Step 3: Replace labels with corresponding numbers
data2['fault'] = data2['fault'].map(label_to_number)

# Now, 'Y' contains numeric labels from 0-9

# You can save the modified DataFrame to a new CSV file if needed
# data2.to_csv('cwru_bearing_numeric_labels.csv', index=False)
# Standardize the data

X2 = data2.iloc[:, 0:9].values  # Features
y2 = data2.iloc[:, 9].values  # Labels

# Standardize the data
scaler = StandardScaler()
X2 = scaler.fit_transform(X2)

In [16]:
#for reading dataset-1
# Load your CSV data1 using pandas
data1 = pd.read_csv('classData.csv')

# Combine the values of the first 4 columns into a new column
data1['Combination'] = data1.apply(lambda row: ''.join(map(str, row[:4])), axis=1)

# Find unique combinations and assign label numbers
unique_combinations = data1['Combination'].unique()
label_mapping = {combination: label for label, combination in enumerate(unique_combinations)}

# Add a new column with the label numbers
data1['Label'] = data1['Combination'].map(label_mapping)

X1 = data1.iloc[:, 4:10].values  # Features
y1 = data1.iloc[:, 11:].values  # Labels

# Standardize the data
scaler = StandardScaler()
X1 = scaler.fit_transform(X1)

# One-hot encode the labels
# y1_encoded = to_categorical(y1, num_classes=6)
num_classes=6
y1_encoded = y1

In [17]:

# Define a simple neural network model
def create_model(input_shape, num_classes, num_layers):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(64, activation='relu', input_shape=input_shape))
    
    for _ in range(num_layers):
        model.add(tf.keras.layers.Dense(32, activation='relu'))
    
    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    
    return model

# Define the fitness function
def fitness_function(model, X_train, y_train, X_test, y_test):
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, np.argmax(y_pred, axis=1))
    return accuracy

# Generate initial population with different layer combinations
def generate_initial_population(input_shape, num_classes, max_layers, population_size):
    population = []
    for _ in range(population_size):
        num_layers = random.randint(1, max_layers)
        model = create_model(input_shape, num_classes, num_layers)
        population.append(model)
    return population

# Evolutionary algorithm to optimize layers and weights
def evolve_population(population, X_train, y_train, X_test, y_test, num_generations, mutation_rate,input_shape):
    for generation in range(num_generations):
        fitness_scores = [fitness_function(model, X_train, y_train, X_test, y_test) for model in population]
        
        # Select the top-performing models (elitism)
        num_parents = 3
        parents = np.argsort(fitness_scores)[-num_parents:]
        
        # Crossover and mutation
        new_population = []
        for i in range(len(population)):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            
            # Crossover: Take layers from parents
            num_layers = population[parent1].count_params()
            child_model = create_model(input_shape, num_classes, num_layers)
            
            # Mutation: Randomly add or remove layers
            if random.random() < mutation_rate:
                num_mutations = random.randint(-1, 1)  # Adjust the range as needed
                num_layers += num_mutations
            
            new_population.append(create_model(input_shape, num_classes, num_layers))
        
        population = new_population
    
    # Return the best model from the final generation
    best_model = population[np.argmax(fitness_scores)]
    best_accuracy = fitness_scores[np.argmax(fitness_scores)]
    return best_model, best_accuracy


In [20]:
#Compiling and testing model for dataset-1

#Splitting dataset-1 into train and test
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1_encoded, test_size=0.2, random_state=42)

input_shape1=(X_train1.shape[1],)
num_classes1=6
max_layers1 = 4
population_size1 = 10

initial_population1 = generate_initial_population(input_shape1, num_classes1, max_layers1, population_size1)
best_model1, best_accuracy1 = evolve_population(initial_population1, X_train1, y_train1, X_test1, y_test1, 10, 0.1,input_shape1)


50/50 [==============================] - 0s 836us/step


RecursionError: maximum recursion depth exceeded

In [ ]:
#Compiling and testing model for dataset-2

#Splitting dataset-1 into train and test
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)
num_classes2=10
input_shape2=(X_train2.shape[1],)
max_layers2 = 4
population_size2 = 10

initial_population2 = generate_initial_population(input_shape2, num_classes2, max_layers2, population_size2)
best_model2, best_accuracy2 = evolve_population(initial_population2, X_train2, y_train2, X_test2, y_test2, 10, 0.1,input_shape2)


In [ ]:

# Print best accuracies
print(f"Best accuracy for Dataset-1: {best_accuracy1}")
print(f"Best accuracy for Dataset-2: {best_accuracy2}")
